## Screening Code

This code can only run one stimulation at a time but it will run and record every stimulation you run!

Run this to import necessary libraries and reset the recording counter

In [1]:
#import libraries
import pysmu
from pysmu import Session, Mode, Device
import time
import pandas as pd
import matplotlib.pyplot as plt
#initialize the session run count to keep recordings in order
run = 1

Run this to start the ADALM session. If something goes wrong and you need to re run this code, you will have to refresh your kernal before re-initializing the session
From this point on in this session, only re run the code below this initialization step

In [2]:
# initialize ADALM settings
session = Session(queue_size = 100000)
sample_rate = 100000
session.configure(sample_rate = sample_rate)

Edit the code below to choose the parameters to apply for this stimulation

It will calculate duty cycle for you automatically

Don't change the labels of the paramters

In [3]:
# uncomment to set mode to current or voltage stimulation 
# mode = "current"
mode = "voltage"

# set parameters below
magnitude = 5 #units: V for voltage stim or A for current stim

pulse_train_length = 1 #unit: seconds


Choose the channel applying the stimulation, I always use channel A

In [4]:
#load stim conditions 
channel = session.devices[0].channels['A']

Test to see if your circuit is connected

In [6]:
#Test for connection
n = 100

channel.mode = Mode.SVMI
channel.constant(0.1)
recording = channel.get_samples(n)
# print(recording)
currents = []
for number in range(n):
    currents.append(recording[number][1])
avg_current = sum(currents)/len(currents)
print(avg_current)
if avg_current > 0.01:
    print(f"Channel is shorted")
elif avg_current < 0.0002:
    print(f"Channel is disconnected")
else:
    print(f"Channel is connected")
channel.constant(0)
channel.get_samples(10)
print("complete")

0.0009746701497351751
Channel is connected
complete


This code runs the stimulation

Every time it is run, it will apply the stimulation specificed above and will save the V and A traces to a file in the same folder as this code named according to the parameters of the stimulation

In [7]:
# set the channel to current or voltage stim
if mode == "current":
    channel.mode = Mode.SIMV
elif mode == "voltage":
    channel.mode = Mode.SVMI
    
#apply stimulation
print("stim")
channel.constant(magnitude)
recording = channel.get_samples(sample_rate*pulse_train_length)

#save V and A recordings
V = []
A = []
for i in range (sample_rate*pulse_train_length):
    V.append(recording[i][0])
    A.append(recording[i][1])
df = pd.DataFrame({"V" : V, "A": A,})
if mode == "voltage":
    df.to_csv(f"{magnitude}V_{run}.csv")
elif mode == "current":
    df.to_csv(f"{magnitude}A_{run}.csv")
run += 1
time.sleep(1)


stim
